<a href="https://colab.research.google.com/github/jovanadobreva/Labs-I2DS/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

### Import the required libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler,MinMaxScaler
from sklearn.impute import SimpleImputer
import missingno as msno
from sklearn.model_selection import train_test_split, GridSearchCV,KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor


### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [2]:
data = pd.read_csv('ElectricCarData.csv')

In [3]:
data

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,Nissan,Ariya 63kWh,7.5,160,330,191,440,Yes,FWD,Type 2 CCS,Hatchback,C,5,45000
99,Audi,e-tron S Sportback 55 quattro,4.5,210,335,258,540,Yes,AWD,Type 2 CCS,SUV,E,5,96050
100,Nissan,Ariya e-4ORCE 63kWh,5.9,200,325,194,440,Yes,AWD,Type 2 CCS,Hatchback,C,5,50000
101,Nissan,Ariya e-4ORCE 87kWh Performance,5.1,200,375,232,450,Yes,AWD,Type 2 CCS,Hatchback,C,5,65000


### Encode string variables

In [172]:
data.isnull().any().sum()

0

In [225]:
def encode(data, columns):
    encoder = LabelEncoder()
    copy = data.copy()
    for column in columns:
        copy[column] = encoder.fit_transform(copy[column].astype(str))
        if 'nan' in encoder.classes_:
            copy.loc[copy[column] == copy[column].max(), column] = np.nan
    return copy

df = encode(data,['Brand','Model','RapidCharge','PowerTrain','PlugType','BodyStyle','Segment'])
df.dtypes

Brand                int32
Model                int32
AccelSec           float64
TopSpeed_KmH         int64
Range_Km             int64
Efficiency_WhKm      int64
FastCharge_KmH      object
RapidCharge          int32
PowerTrain           int32
PlugType             int32
BodyStyle            int32
Segment              int32
Seats                int64
PriceEuro            int64
dtype: object

In [226]:
df['FastCharge_KmH'] = df['FastCharge_KmH'].replace('-',df['FastCharge_KmH'].mode()[0])
df['FastCharge_KmH'] = df['FastCharge_KmH'].astype('int')
df.dtypes


Brand                int32
Model                int32
AccelSec           float64
TopSpeed_KmH         int64
Range_Km             int64
Efficiency_WhKm      int64
FastCharge_KmH       int32
RapidCharge          int32
PowerTrain           int32
PlugType             int32
BodyStyle            int32
Segment              int32
Seats                int64
PriceEuro            int64
dtype: object

In [175]:
df['FastCharge_KmH'].mode()[0]

230

## Split the dataset for training and testing in ratio 80:20

In [241]:
X = df.drop(columns=['PriceEuro'],axis=1)
Y = df['PriceEuro']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
scaler= StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

In [228]:
model = DecisionTreeRegressor()

param_grid = {
    'max_depth': [3, 5, 10, 15, 20,30],
    'min_samples_split': [2, 5, 7, 10],
    'max_features': ['sqrt', 'log2', None]
}

kfold = KFold(n_splits=10, random_state=0, shuffle=True)
gs = GridSearchCV(estimator=model, param_grid=param_grid, cv=kfold, scoring='neg_mean_squared_error', n_jobs=-1)
gs.fit(X_train, Y_train)

gs.best_estimator_

DecisionTreeRegressor(max_depth=5, max_features='log2', min_samples_split=5)

Fit the model using the fit function


## Predict the outcomes for X test

In [229]:
y_predTree = gs.best_estimator_.predict(X_test)


## Assess the model performance, by using sklearn metrics for regression

In [230]:
r2_score(Y_test,y_predTree)

0.7587032046340237

## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [245]:
model = XGBRegressor()

param_grid = {
    'objective': ['reg:squarederror'],
    'n_estimators': [50, 100, 150,200], 
    'max_depth': [3, 6, 10],         
    'learning_rate': [0.001, 0.01, 0.1,0,2, 0.3] 
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=kfold, n_jobs=-1)
grid_search.fit(X_train, Y_train)

grid_search.best_estimator_

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## Predict the outcomes for X test

In [246]:
y_predXGB = grid_search.best_estimator_.predict(X_test)

## Assess the model performance, by using sklearn metrics for regression

In [247]:
r2_score(Y_test,y_predXGB)

0.45782821878040114

## Compare the performances of both model for at least three regression metircs

In [237]:

maeTree = mean_absolute_error(Y_test, y_predTree)
mseTree = mean_squared_error(Y_test, y_predTree)
r2Tree = r2_score(Y_test, y_predTree)

range = df['PriceEuro'].max() - df['PriceEuro'].min()

print(f'Relative Mean Absolute Error (MAE): {(maeTree / range) * 100}')
print(f'Relative Mean Squared Error (MSE): {(mseTree / (range * range)) * 100}')
print(f'R-squared (R²): {r2Tree}')

maeXGB = mean_absolute_error(Y_test, y_predXGB)
mseXGB = mean_squared_error(Y_test, y_predXGB)
r2XGB = r2_score(Y_test, y_predXGB)

range = df['PriceEuro'].max() - df['PriceEuro'].min()

print(f'Relative Mean Absolute Error (MAE): {(maeXGB / range) * 100}')
print(f'Relative Mean Squared Error (MSE): {(mseXGB / (range * range)) * 100}')
print(f'R-squared (R²): {r2XGB}')

Relative Mean Absolute Error (MAE): 6.907092438495263
Relative Mean Squared Error (MSE): 1.6167441494788517
R-squared (R²): 0.7587032046340237
Relative Mean Absolute Error (MAE): 10.466851441680223
Relative Mean Squared Error (MSE): 3.632675908396716
R-squared (R²): 0.45782821878040114
